# Specialize For SVJAIR

In [17]:
import pandas as pd
import time
import csv
import requests
import datetime
import json
import os

class GetSjvAir:

    #initialize GetSjvAir --> inputs are self, list_sensor,start_date, end_date, data
    
    #list_sensor --> list of sensor ID's / names
    #start_date / end_date --> can just do date in this format: YYYY-MM-DD
    #you can also add time by putting it next to the date in this format --> 00:00:00 (24 hour format)
    #data --> list of data types you want (ex: timestamp, celsius, pressure, etc.)
    
    def __init__(self , list_sensor , start_date , end_date , data): 
        self.list_sensor = list_sensor
        self.start_date = start_date
        self.end_date = end_date
        self.data = data        
    
    #inputs are name --> ID name for sensor
    #takes the name of the sensor finds out if the user wanted static information like the sensor name or coordinates
    def get_data_json_sjvair(self , name):
        
        response = requests.get(f"https://www.sjvair.com/api/1.0/monitors/{name}")
        
        sensor_data = response.json()
        
        result_list = {}
        
        #if the user wants the name given to the sensor
        if "name" in self.data:
            
            result_list = {
                'sensor_id': name,
                'name': sensor_data['data']['name'],
            }
        
        #if the user wants the coordinates of the sensor
        elif "log" in self.data or "lat" in self.data:
            result_list = {
                'sensor_id': name,
                'log': sensor_data['data']['position']['coordinates'][0],
                'lat': sensor_data['data']['position']['coordinates'][1],
            }
        
        return result_list
    

    #SJVAir doens't give the option to choose what type of data the user wants when getting the CSV data
    #So this function gets the full CSV file and then makes a new one with only the data the user asked for
    def get_data_sjvair(self):
        
        parameters = {'timestamp__gte': self.start_date, 'timestamp__lt': self.end_date}
        
        #if there isn't a directory of this name, then it will make one
        if not os.path.exists("svjair_folder"):
            os.makedirs("svjair_folder")


        for i in self.list_sensor:
            
            all_data = []
            
            csv_response = requests.get(f"https://www.sjvair.com/api/1.0/monitors/{i}/entries/csv/", params=parameters)
            
            with open(f"svjair_folder/SVJAir_{i}.csv", "wb") as f:
                f.write(csv_response.content)

            data_df = pd.read_csv(f"svjair_folder/SVJAir_{i}.csv")
            
            result_list = self.get_data_json_sjvair(name=i)
            
            for j in range(len(data_df)):
                
                #val gets the whole row @ row j
                val = data_df.iloc[j]    
                val_data = {}      
                
                for k in self.data:
                    
                    try:
                        
                        #gets data from current row and adds it to val_data dictionary
                        val_data[k] = val[k]
                        
                    except KeyError:
                        
                        continue
                
                #adds the static data from result_list into val_data
                val_data.update(result_list)
                
                #adds the current row to all data dictionary
                all_data.append(val_data)

            #once all the data that the user wnats is put into all_data, then convert the dictionary to csv file
            df_final = pd.DataFrame(all_data)
            
            df_final.to_csv(f"svjair_folder/SVJAir_{i}.csv", index=False)
            
            print(f"Finish Download svjair_folder/SVJAir_{i}.csv")
            


# Specialize For PurpleAIr

In [16]:
class GetPurpleAirData:
    
    #initialize GetPurpleAirData --> inputs are sensor_list, start_date, end_date, headers, fields

    #sensor_list --> list of ID numbers for the sensors

    #start_date / end_date --> datetime.datetime(YYYY, MM, DD, 00,00,00)
    #you can also do it without adding the time

    #headers --> API key

    #fields: type of data the user wants
    def __init__(self, sensor_list, start_date, end_date, headers, fields):
        self.sensor_list = sensor_list
        self.start_date = int(start_date.timestamp())
        self.end_date = int(end_date.timestamp())
        self.headers = headers
        self.fields = fields
    
    #converts UNIX time into normal timestamp format
    def convert_date_time(self , timestampunix):
    
        pst_timezone = datetime.timezone(datetime.timedelta(hours=-8))
        converted_time = datetime.datetime.fromtimestamp(timestampunix, tz= datetime.timezone.utc)
        converted_time_pst = converted_time.astimezone(pst_timezone)
        converted_time_pst.isoformat()
        return converted_time_pst

    #cleans the returned date from convert_date_time function and puts it into the csv file
    def clean_date(self , file_name):
        
        df = pd.read_csv(file_name)
                    
        df['time_stamp'] = df['time_stamp'].apply(self.convert_date_time)
            
        df = df.sort_values(by='time_stamp', ascending=True)
            
        df.to_csv(file_name, index=False)

    
    def get_data_purpleair(self):
        
        for index in range(len(self.sensor_list)):
            
            url = f'https://api.purpleair.com/v1/sensors/{self.sensor_list[index]}/history/csv?start_timestamp={self.start_date}&end_timestamp={self.end_date}&fields={",".join(self.fields)}'
            
            try:
                
                #get data from API
                response = requests.get(url, headers=self.headers)
                
                response.raise_for_status()

                #if this directory doesn't exist, then it will make it
                if not os.path.exists("purpleair_folder"):
                    os.makedirs("purpleair_folder")
                
                filename = f"purpleair_folder/sensor_{self.sensor_list[index]}_data.csv"
                
                with open(filename , "wb") as f:
                    
                    f.write(response.content)
                    
                print(f"Finish Download {filename}")
                
                #after writing down the data, it will clean the UNIX dates into normal timestamp dates
                self.clean_date(filename)
            
            #prints out errors if they happen
            except requests.HTTPError as http_err:
                
                print(http_err)
                
            except Exception as err:
                
                print(err)


# Main Program

In [19]:

if __name__ == '__main__':
    
    datatypes = input("(SJVAIR OR PurpleAir)")
    
    if datatypes.upper() == 'SJVAIR':
        
        #initialize GetSjvAir --> inputs are self, list_sensor,start_date, end_date, data
    
        #list_sensor --> list of sensor ID's / names
        #start_date / end_date --> datetime.datetime(YYYY, MM, DD, 00,00,00) (last 3 entries is the time)
        #you can also do it without adding the time
        #data --> list of data types you want (ex: timestamp, celsius, pressure, etc.)
        data = GetSjvAir(
            list_sensor=["yiz5lVBeRf26NdX-KtrPlg" , "K6rfX7COTLKuQIFIh00gjA"] , 
            start_date = datetime.datetime(2023,11,16,5,30,0), 
            end_date= datetime.datetime(2023,11,16,7,0,0), 
            data = ['timestamp' , "celsius" , "pm25" , "humidity" , "name" , "pressure"] , 
        )
        
        data.get_data_sjvair()
        
    elif datatypes.upper() == 'PURPLEAIR':
        
        #initialize GetPurpleAirData --> inputs are sensor_list, start_date, end_date, headers, fields

        #sensor_list --> list of ID numbers for the sensors

        #start_date / end_date --> datetime.datetime(YYYY, MM, DD, 00,00,00) (last 3 entries is the time)
        #you can also do it without adding the time

        #headers --> API key

        #fields: type of data the user wants
        data = GetPurpleAirData(
            sensor_list=["155683", "104106"],
            start_date=datetime.datetime(2023, 11, 16,5,30,0),
            end_date=datetime.datetime(2023, 11, 16,8,0,0),
            headers={'X-API-Key': "5E56E968-4C34-11EE-A77F-42010A800009"},
            fields=["pm2.5_alt","pm2.5_atm","pressure","humidity"]
            
        )
        
        data.get_data_purpleair()
        
    else:
        
        print("Please Choose Between PurpleAir or SJVAIR")

Finish Download svjair_folder/SVJAir_yiz5lVBeRf26NdX-KtrPlg.csv
Finish Download svjair_folder/SVJAir_K6rfX7COTLKuQIFIh00gjA.csv
